# Weather: OpenMeteoInput
[Reference](https://velog.io/@jjlee6496/Functions-Tools-and-agents-with-Langchain-5-9ban48au)

In [14]:
latitude=13
longitude=14

In [19]:
BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
# Parameters for the request
params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
}

# Make the request
response = requests.get(BASE_URL, params=params)
    
if response.status_code == 200:
    results = response.json()
    print('results: ', results)
else:
    raise Exception(f"API Request failed with status code: {response.status_code}")

current_utc_time = datetime.datetime.utcnow()
print('current_utc_time: ', current_utc_time)

results:  {'latitude': 13.0, 'longitude': 14.0, 'generationtime_ms': 0.0209808349609375, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 280.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2024-05-21T00:00', '2024-05-21T01:00', '2024-05-21T02:00', '2024-05-21T03:00', '2024-05-21T04:00', '2024-05-21T05:00', '2024-05-21T06:00', '2024-05-21T07:00', '2024-05-21T08:00', '2024-05-21T09:00', '2024-05-21T10:00', '2024-05-21T11:00', '2024-05-21T12:00', '2024-05-21T13:00', '2024-05-21T14:00', '2024-05-21T15:00', '2024-05-21T16:00', '2024-05-21T17:00', '2024-05-21T18:00', '2024-05-21T19:00', '2024-05-21T20:00', '2024-05-21T21:00', '2024-05-21T22:00', '2024-05-21T23:00'], 'temperature_2m': [29.9, 29.1, 28.5, 27.9, 27.4, 27.1, 28.5, 32.8, 36.1, 38.5, 39.7, 41.3, 44.3, 44.7, 44.8, 44.5, 43.5, 41.7, 38.3, 36.2, 34.5, 32.2, 31.1, 30.1]}}
current_utc_time:  2024-05-21 22:50:34.634014


In [20]:
latitude=37
longitude=127

In [21]:
BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
# Parameters for the request
params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
}

# Make the request
response = requests.get(BASE_URL, params=params)
    
if response.status_code == 200:
    results = response.json()
    print('results: ', results)
else:
    raise Exception(f"API Request failed with status code: {response.status_code}")

current_utc_time = datetime.datetime.utcnow()
print('current_utc_time: ', current_utc_time)

results:  {'latitude': 37.0, 'longitude': 127.0, 'generationtime_ms': 0.026941299438476562, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 5.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2024-05-21T00:00', '2024-05-21T01:00', '2024-05-21T02:00', '2024-05-21T03:00', '2024-05-21T04:00', '2024-05-21T05:00', '2024-05-21T06:00', '2024-05-21T07:00', '2024-05-21T08:00', '2024-05-21T09:00', '2024-05-21T10:00', '2024-05-21T11:00', '2024-05-21T12:00', '2024-05-21T13:00', '2024-05-21T14:00', '2024-05-21T15:00', '2024-05-21T16:00', '2024-05-21T17:00', '2024-05-21T18:00', '2024-05-21T19:00', '2024-05-21T20:00', '2024-05-21T21:00', '2024-05-21T22:00', '2024-05-21T23:00'], 'temperature_2m': [20.5, 23.0, 24.9, 26.5, 27.5, 28.0, 28.0, 27.9, 27.2, 24.9, 19.4, 16.5, 15.1, 14.6, 14.4, 14.4, 14.2, 14.2, 14.2, 14.3, 14.3, 14.4, 14.4, 14.5]}}
current_utc_time:  2024-05-21 22:51:42.765490


In [18]:
from pytz import timezone
format = "%Y-%m-%d %H:%M:%S"
timestr = datetime.datetime.now(timezone('Asia/Seoul')).strftime(format)
print('timestr: ', timestr)

timestr:  2024-05-22 07:50:02


In [22]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

# @tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
        print('results: ', results)
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

# 서울시 서초구 ; 127.0348111, 37.48078611

In [23]:
get_current_temperature(longitude=127.0348111, latitude=37.48078611)

results:  {'latitude': 37.5, 'longitude': 127.0625, 'generationtime_ms': 0.025987625122070312, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 75.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2024-05-21T00:00', '2024-05-21T01:00', '2024-05-21T02:00', '2024-05-21T03:00', '2024-05-21T04:00', '2024-05-21T05:00', '2024-05-21T06:00', '2024-05-21T07:00', '2024-05-21T08:00', '2024-05-21T09:00', '2024-05-21T10:00', '2024-05-21T11:00', '2024-05-21T12:00', '2024-05-21T13:00', '2024-05-21T14:00', '2024-05-21T15:00', '2024-05-21T16:00', '2024-05-21T17:00', '2024-05-21T18:00', '2024-05-21T19:00', '2024-05-21T20:00', '2024-05-21T21:00', '2024-05-21T22:00', '2024-05-21T23:00'], 'temperature_2m': [19.0, 21.0, 22.8, 24.1, 25.3, 26.0, 26.4, 26.6, 26.5, 26.0, 23.0, 21.2, 18.8, 17.0, 15.8, 15.1, 14.6, 14.3, 14.2, 14.2, 14.2, 14.2, 14.4, 14.6]}}


'The current temperature is 14.6°C'

## 결과를 보면 현재 기준 18도 여야하므로 맞지 않음